In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from transformers import BertTokenizer, BertModel
from torch.utils.data import Dataset, DataLoader
from fpdf import FPDF

c:\Users\Tempo\anaconda3\envs\meal_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Dataset
dataset_path = "data/daily_food_nutrition_dataset.csv"
df = pd.read_csv(dataset_path)
df

,Date,User_ID,Food_Item,Category,Calories (kcal),Protein (g),Carbohydrates (g),Fat (g),Fiber (g),Sugars (g),Sodium (mg),Cholesterol (mg),Meal_Type,Water_Intake (ml)
0,2024-09-11,496,Eggs,Meat,173,42.4,83.7,1.5,1.5,12.7,752,125,Lunch,478
1,2024-12-17,201,Apple,Fruits,66,39.2,13.8,3.2,2.6,12.2,680,97,Lunch,466
2,2024-06-09,776,Chicken Breast,Meat,226,27.1,79.1,25.8,3.2,44.7,295,157,Breakfast,635
3,2024-08-27,112,Banana,Fruits,116,43.4,47.1,16.1,6.5,44.1,307,13,Snack,379
4,2024-07-28,622,Banana,Fruits,500,33.9,75.8,47.0,7.8,19.4,358,148,Lunch,471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2024-09-18,455,Salmon,Meat,346,29.8,55.6,4.6,1.7,0.5,976,87,Breakfast,793
9996,2024-12-13,913,Grapes,Fruits,174,22.9,54.9,32.1,2.5,5.9,255,56,Lunch,665
9997,2024-01-31,943,Strawberry,Fruits,63,36.5,23.8,21.6,0.8,48.9,757,63,Snack,876
9998,2024-09-28,571,Spinach,Vegetables,564,26.2,58.9,11.9,3.3,43.0,482,33,Breakfast,319


In [3]:
# ---------------- 1. CNN Model for Food Recognition ----------------
class FoodCNN(nn.Module):
    def __init__(self):
        super(FoodCNN, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, len(df['Food_Item'].unique()))
    
    def forward(self, x):
        return self.model(x)



In [4]:
# Instantiate CNN Model
cnn_model = FoodCNN()

c:\Users\Tempo\anaconda3\envs\meal_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tempo\anaconda3\envs\meal_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Tempo/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [05:34<00:00, 140kB/s] 


In [46]:
# Save the trained CNN model

torch.save(cnn_model.state_dict(), "models/food_cnn_model.pth")
print("CNN Model Saved!")

CNN Model Saved!


In [5]:
# ---------------- 2. BERT for Recipe Understanding ----------------
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertModel.from_pretrained("bert-base-uncased")

class RecipeDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts
        self.encodings = tokenizer(self.texts, padding=True, truncation=True, return_tensors='pt')
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encodings.items()}

In [6]:
# Example Recipes Dataset
recipes = ["Scrambled eggs with toast", "Chicken and rice", "Salad with dressing"]
recipe_dataset = RecipeDataset(recipes)
recipe_loader = DataLoader(recipe_dataset, batch_size=2)

In [7]:
# ---------------- 3. LSTM for Meal History Tracking ----------------
class LSTMMealTracker(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMMealTracker, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

lstm_model = LSTMMealTracker(input_size=5, hidden_size=10, output_size=3)


In [47]:
# Save the trained LSTM model
torch.save(lstm_model.state_dict(), "models/meal_tracker_lstm.pth")
print("LSTM Model Saved!")

LSTM Model Saved!


In [10]:
# ---------------- 4. Budget-Based Meal Planning ----------------
def filter_meals_by_budget(user_id, max_budget):
    user_meals = df[df['User_ID'] == user_id]
    affordable_meals = user_meals[user_meals['Cost'] <= max_budget]
    return affordable_meals.head(7)  # Weekly Plan

In [36]:
# ---------------- 5. Ingredient Substitution for Allergies ----------------
def substitute_allergenic_ingredients(user_id, allergy):
    if 'Food_Item' in df.columns:
        user_meals = df[df['User_ID'] == user_id]
        for idx, row in user_meals.iterrows():
            if allergy.lower() in row['Food_Item'].lower():
                user_meals.at[idx, 'Food_Item'] = "Alternative Food Option"
        return user_meals.head(7)
    else:
        print("Warning: 'Food_Item' column missing in dataset.")
        return df[df['User_ID'] == user_id].head(7)



In [37]:
# ---------------- 6. Seasonal Meal Suggestions ----------------
def get_seasonal_meals(season):
    seasonal_meals = df[df['Season'] == season]
    return seasonal_meals.sample(n=7) if not seasonal_meals.empty else df.sample(n=7)

In [29]:
# ---------------- 7. Real-Time Grocery Pricing ----------------
# def estimate_meal_cost(meal):
#     # Mock pricing mechanism (real implementation would use API calls)
#     avg_price = df[df['Food_Item'] == meal]['Cost'].mean()
#     return avg_price if not pd.isna(avg_price) else 5.0  # Default price


In [39]:
# ---------------- 8. Smart Grocery List Generation ----------------
def generate_grocery_list(meal_plan):
    grocery_list = {}
    if 'Food_Item' in meal_plan.columns:
        for _, row in meal_plan.iterrows():
            food_item = row['Food_Item']
            grocery_list[food_item] = grocery_list.get(food_item, 0) + 1
    else:
        print("Warning: 'Food_Item' column missing in dataset.")
    return grocery_list


In [40]:
# ---------------- 9. Meal Prep Tips ----------------
def get_meal_prep_tips(meal):
    return f"Tips for preparing {meal}: Ensure proper seasoning and balanced cooking time."


In [43]:
# ---------------- 10. PDF Meal Plan Generation ----------------
def generate_meal_plan(user_id, allergy=None, season=None):
    if allergy:
        user_meals = substitute_allergenic_ingredients(user_id, allergy)
    elif season:
        user_meals = get_seasonal_meals(season)
    else:
        user_meals = df[df['User_ID'] == user_id].head(7)
    
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=12)
    pdf.cell(200, 10, txt=f"Meal Plan for User {user_id}", ln=True, align='C')
    
    grocery_list = generate_grocery_list(user_meals)
    
    for _, row in user_meals.iterrows():
        meal_prep_tips = get_meal_prep_tips(row['Food_Item'])
        pdf.cell(200, 10, txt=f"{row['Date']}: {row['Meal_Type']} - {row['Food_Item']} ({row['Calories (kcal)']} kcal", ln=True)
        pdf.multi_cell(200, 10, txt=meal_prep_tips)
    
    pdf.add_page()
    pdf.cell(200, 10, txt="Grocery List", ln=True, align='C')
    for item, qty in grocery_list.items():
        pdf.cell(200, 10, txt=f"{item}: {qty}", ln=True)
    
    pdf.output("meal_plan.pdf")
    print("Meal Plan Generated with Grocery List and Meal Prep Tips!")




In [44]:
# Example: Generate a Meal Plan
generate_meal_plan(user_id=201)

Meal Plan Generated with Grocery List and Meal Prep Tips!


### Loading the CNN Model

In [ ]:
#Using model

# Reinitialize the model
cnn_model = FoodCNN()
cnn_model.load_state_dict(torch.load("food_cnn_model.pth"))
cnn_model.eval()  # Set the model to evaluation mode
print("CNN Model Loaded!")

### Loading the LSTM Model

In [ ]:
lstm_model = LSTMMealTracker(input_size=5, hidden_size=10, output_size=3)
lstm_model.load_state_dict(torch.load("meal_tracker_lstm.pth"))
lstm_model.eval()
print("LSTM Model Loaded!")
